In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

# Get the rankings for each country

## Get Alexa rankings for one country

In [5]:
from funcy import compose

def get_rankings (country_ranking_url, country_name):
        
    def parse_ranking (country_ranking_url):
        '''Takes a country ranking URL and returns a list of (bs4 parsed) HTML ranking listings.'''
        rankings_html = requests.get(country_ranking_url).content
        soup = BeautifulSoup(rankings_html, 'html.parser')
        listings = soup.find_all("div", {"class": "site-listing"})
        return listings

    def extract_info (site_listing_html):
        '''Takes HTML of an Alexa site listing and returns JSON.'''
        attrs = site_listing_html.find_all("div", {"class":"td"})
        rank = int(attrs[0].text)
        url = attrs[1].a.text
        site_info = attrs[1].a['href']
        return {
            'rank': rank,
            'url': url,
            'site_info': site_info,
            'country_name': country_name,
        }

    return [extract_info(ranking) for
           ranking in parse_ranking(country_ranking_url)]        
    

get_rankings('https://www.alexa.com/topsites/countries/AF', 'Afghanistan')


[]

## Get name and URL for all the countries

In [7]:
top = get_rankings('https://www.alexa.com/topsites', '')

In [10]:
top_urls = [s['url'] for s in top]

In [20]:
tlds = [u.split('.')[-1] for u in top_urls]

In [18]:
providers = pd.read_csv('providers_labeled - providers_unlabeled.csv')
def get_juris (tld):
    return providers[providers['name']==tld]['country (alpha2)'].values[0]
get_juris('.com') 

'US'

In [53]:
tld_jurisdictions = [get_juris(f'.{tld}') for tld in tlds]

In [57]:
len([j for j in tld_jurisdictions if j == 'US'])/len(tld_jurisdictions)

0.8